## 1. 필수 라이브러리 임포트 및 환경 설정
- .env에 openai api key 저장: sk-.....

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
# from langchain.llms import OpenAI
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableParallel, RunnableMap, RunnablePassthrough, RunnableLambda
from dotenv import load_dotenv
from langchain_teddynote import logging
from langchain_teddynote.messages import stream_response

# env 파일 읽기
load_dotenv()

# OpenAI API 키 가져오기
openai_api_key = os.getenv("OPENAI_API_KEY")

# 프로젝트 이름을 입력합니다. 
# - 로깅용 내부망에서 안됨
# - Prompt, Output에 대한 token 사용량 및 과금 정보 추적 가능
# logging.langsmith("AI_report", set_enable=True)

## 2. Sample Data 생성
- postgre와 연결 필요

In [ ]:
import os
import pandas as pd
import numpy as np

# Random seed 설정 for reproducibility
np.random.seed(42)

# 회사 및 업종 정보 생성
num_companies = 100  # 총 100개의 회사 생성
company_names = [f'Company_{i+1}' for i in range(num_companies)]  # 회사명 리스트 생성
industries = ['Manufacturing', 'Retail', 'Technology', 'Healthcare', 'Finance']  # 업종 리스트

# 각 회사에 대한 고정된 업종 리스트 생성
company_industries = [np.random.choice(industries) for _ in range(num_companies)]  # 각 회사의 업종을 무작위로 할당

# 날짜 생성 (2021-01 ~ 2024-09)
dates = pd.date_range(start="2020-01-01", end="2024-09-01", freq='MS').strftime("%Y-%m").tolist()  # 월별 날짜 리스트 생성

# 기본 정보 데이터 생성 함수 정의
def generate_basic_data(company, industry, date):
    base_asset_value = 30000  # 기본 총자산 값 설정
    total_assets = round(base_asset_value * (1 + np.random.normal(0, 0.05)), 0)  # 기본 총자산 값에서 5% 변동 (정규분포 이용, 소수점 없이)
    revenue = round(np.random.uniform(1000, 10000), 0)  # 1천만원 ~ 1억원 사이의 매출액 값 설정 (소수점 없이)
    operating_profit = round(revenue * np.random.uniform(-0.2, 0.2), 0)  # 영업이익 = 매출액의 -20%~20%로 계산 (소수점 없이)
    net_income = round(operating_profit * np.random.uniform(-0.5, 0.8), 0)  # 당기순이익 = 영업이익의 -50%~80%로 계산 (소수점 없이)
    short_term_loans = round(np.random.uniform(500, 5000), 0)  # 단기차입금 (500만원 ~ 5000만원 사이, 소수점 없이)
    long_term_loans = round(np.random.uniform(1000, 10000), 0)  # 장기차입금 (1000만원 ~ 1억원 사이, 소수점 없이)
    total_loans = short_term_loans + long_term_loans  # 총 차입금 계산
    loan_to_sales = round((total_loans / revenue) * 100, 2)  # 매출대비차입금 비율
    working_capital_turnover = round(np.random.uniform(1, 5), 2)  # 운전자금회전율 (1~5회)
    operating_cash_flow = round(np.random.uniform(500, 5000), 0)  # 영업활동현금흐름 (500만원 ~ 5000만원 사이, 소수점 없이)
    net_cash_flow = round(operating_cash_flow - total_loans, 0)  # 순현금흐름 (영업활동현금흐름 - 총 차입금, 소수점 없이)
    ar_balance = round(np.random.uniform(1000, 5000), 0)  # 월별 매출채권 규모 (1000만원 ~ 5000만원, 소수점 없이)
    ar_collection_period = round(np.random.uniform(30, 90), 0)  # 월별 매출채권 회수기일 (30일 ~ 90일, 소수점 없이)
    ap_balance = round(np.random.uniform(800, 4000), 0)  # 월별 매입채무 규모 (800만원 ~ 4000만원, 소수점 없이)
    ap_payment_period = round(np.random.uniform(20, 80), 0)  # 월별 매입채무 회수기일 (20일 ~ 80일, 소수점 없이)
    
    return {
        '기업명': company,
        '업종': industry,
        '날짜': date,
        # 성장성
        '매출액증가율': round(np.random.uniform(5, 15), 2),  # 5~15% 사이의 매출액증가율 값
        '총자산증가율': round(np.random.uniform(3, 10), 2),  # 3~10% 사이의 총자산증가율 값
        '총자산': int(total_assets),  # 소수점 없이 월별 정수형 총자산 값
        '매출액': int(revenue),  # 소수점 없이 정수형 월별 매출액 값
        # 수익성
        '영업이익': int(operating_profit),  # 영업이익 추가 (정수형)
        '영업이익률': round(operating_profit / revenue * 100, 2) if revenue != 0 else round(np.random.uniform(-20, 20), 2),  # 영업이익률 추가 (음수 가능)
        '당기순이익': int(net_income),  # 당기순이익 추가 (정수형)
        '당기순이익률': round(net_income / revenue * 100, 2) if revenue != 0 else round(np.random.uniform(-30, 30), 2),  # 당기순이익률 추가 (음수 가능)
        # 재무 안정성
        '단기차입금': int(short_term_loans),  # 단기차입금 추가
        '장기차입금': int(long_term_loans),  # 장기차입금 추가
        '매출대비차입금': loan_to_sales,  # 매출대비차입금 비율 추가
        '운전자금회전율': working_capital_turnover,  # 운전자금회전율 추가
        # 인적 관리
        '인원수': np.random.randint(50, 300),  # 50~300명 사이의 임직원 수 설정
        '월평균급여액': round(np.random.uniform(200, 500), 1),  # 200~500만원 사이의 월평균 급여액 (소수점 없이)
        '월매출창출액': round(np.random.uniform(5000, 20000), 1),  # 5000~20000만원 사이의 월 매출 창출액 (소수점 없이)
        # 현금흐름
        '영업활동현금흐름/매출액': round(np.random.uniform(0.05, 0.2), 2),  # 5~20% 사이의 영업활동현금흐름/매출액 비율
        '영업활동현금흐름': int(operating_cash_flow),  # 영업활동현금흐름 추가
        '순현금흐름': int(net_cash_flow),  # 순현금흐름 추가
        # 거래처 안정성
        '월별_매출채권_규모': int(ar_balance),  # 월별 매출채권 규모
        '월별_매출채권_회수기일': int(ar_collection_period),  # 월별 매출채권 회수기일
        '월별_매입채무_규모': int(ap_balance),  # 월별 매입채무 규모
        '월별_매입채무_회수기일': int(ap_payment_period),  # 월별 매입채무 회수기일
    }

# 기본 정보 데이터 생성
basic_data = [
    generate_basic_data(company, industry, date)  # 각 회사와 날짜별로 기본 정보를 생성
    for company, industry in zip(company_names, company_industries)
    for date in dates
]

# 기본 정보 데이터프레임 생성
df_company_info = pd.DataFrame(basic_data)  # 회사 기본 정보로 데이터프레임 생성

# 데이터프레임 출력
print(df_company_info)

# 거래처 정보 데이터 생성 함수 정의
def generate_partner_data(company, date):
    # 상위 5개 매출처 및 매입처 정보 무작위로 생성
    sales_companies = list(np.random.choice(sales_partner_pool, 5, replace=False))
    purchase_companies = list(np.random.choice(purchase_partner_pool, 5, replace=False))

    # 각 매출처 및 매입처의 신용등급을 무작위로 선택
    sales_grades = list(np.random.choice(['AAA', 'AA+', 'AA', 'A+', 'A', 'BBB+', 'BBB', 'BB', 'B', 'CCC-'], 5, replace=False))
    purchase_grades = list(np.random.choice(['AAA', 'AA+', 'AA', 'A+', 'A', 'BBB+', 'BBB', 'BB', 'B', 'CCC-'], 5, replace=False))

    # 매출처 및 매입처의 집중도를 Dirichlet 분포를 이용해 생성하고, 총합이 100%가 되도록 정규화
    sales_concentrations = list(np.round(np.random.dirichlet(np.ones(5), 1)[0] * 100, 2))
    sales_concentrations = list(np.round(np.array(sales_concentrations) / sum(sales_concentrations) * 100, 2))

    purchase_concentrations = list(np.round(np.random.dirichlet(np.ones(5), 1)[0] * 100, 2))
    purchase_concentrations = list(np.round(np.array(purchase_concentrations) / sum(purchase_concentrations) * 100, 2))

    # 매출처 및 매입처 정보 반환 (각 회사 당 5개)
    return [
        {
            '기업명': company,
            '날짜': date,
            '매출처_회사명': sales_companies[i],
            '매출처_신용등급': sales_grades[i],
            '매출처_집중도': sales_concentrations[i],
            '매입처_회사명': purchase_companies[i],
            '매입처_신용등급': purchase_grades[i],
            '매입처_집중도': purchase_concentrations[i]
        }
        for i in range(5)
    ]

# 회사명 리스트에서 중복을 최소화하며 선택할 수 있도록 셋 생성
sales_partner_pool = list(set(f'SalesPartner_{i+1}' for i in range(200)))  # 매출처 Pool 생성 (200개)
purchase_partner_pool = list(set(f'PurchasePartner_{i+1}' for i in range(200)))  # 매입처 Pool 생성 (200개)

# 거래처 정보 데이터 생성
partner_data = [
    partner  # 각 회사와 날짜별로 거래처 정보 생성
    for company in company_names
    for date in dates
    for partner in generate_partner_data(company, date)
]

# 거래처 정보 데이터프레임 생성
df_partner_info = pd.DataFrame(partner_data)  # 거래처 정보로 데이터프레임 생성

# 샘플 출력
print(df_company_info.head())  # 회사 기본 정보 샘플 출력
print(df_partner_info.head())  # 거래처 정보 샘플 출력

In [7]:
sample = df_company_info[df_company_info['기업명']== 'Company_1']

In [47]:
import pandas as pd
import numpy as np
import pprint # 출력 예시

# 재현성을 위한 Random seed 설정
np.random.seed(42)


def preprocess_growth_data(df_company_info):
    """
    성장성 지표 데이터를 전처리하여 필요한 JSON 형식의 구조로 변환하는 함수.
    
    Args:
        df_company_info (DataFrame): 회사 재무 정보 데이터프레임
    
    Returns:
        dict: 전처리된 성장성 지표 데이터 (JSON 형태)
    """
    # 최신 년월 추출: 데이터프레임에서 가장 최신의 '날짜' 값을 가져옴
    latest_year_month = df_company_info['날짜'].max()
    latest_year = int(latest_year_month.split('-')[0])  # 최신 년도 추출
    latest_month = int(latest_year_month.split('-')[1])  # 최신 월 추출

    # 결과를 저장할 데이터 딕셔너리 정의
    growth_data = {
        'latest_year_month': latest_year_month,  # 최신 년월 기록
        'year_level_data': {},  # 연간 수준 데이터
        'year_rate_data': {},  # 연간 증가율 데이터
        'recent_data': {}  # 최근 12개월 데이터
    }

    # 'year_level_data' 및 'year_rate_data'에 대해 과거 데이터를 추출
    num_years = 3  # 추출할 과거 연도의 수, 필요에 따라 변경 가능
    for year_offset in range(1, num_years + 1):
        target_year = latest_year - year_offset
        year_key = f"{target_year}년"
        december_date = f"{target_year}-12"
        
        # 해당 년도 12월의 데이터를 필터링
        december_data = df_company_info[df_company_info['날짜'] == december_date]
        annual_data = df_company_info[df_company_info['날짜'].str.startswith(f"{target_year}")]
        
        # 12월 데이터와 연간 데이터를 모두 사용할 수 있는 경우
        if not december_data.empty and not annual_data.empty:
            # '성장성' 지표 계산
            total_assets = int(round(december_data.at[december_data.index[0], '총자산']))  # 해당 연도의 총자산 (12월 기준)
            revenue = int(round(annual_data['매출액'].sum()))  # 해당 연도의 전체 매출액의 합계
            
            # 총자산증가율 계산: 이전 연도의 총자산 데이터가 있을 때만
            if year_offset <= num_years:
                prev_year_december_date = f"{target_year - 1}-12"
                prev_december_data = df_company_info[df_company_info['날짜'] == prev_year_december_date]
                if not prev_december_data.empty:
                    prev_total_assets = prev_december_data.at[prev_december_data.index[0], '총자산']
                    asset_growth_rate = round(((total_assets - prev_total_assets) / prev_total_assets) * 100, 2)
                else:
                    asset_growth_rate = 0
            else:
                asset_growth_rate = 0

            # 매출액증가율 계산: 이전 연도의 매출 데이터가 있을 때만
            if year_offset <= num_years:
                prev_annual_data = df_company_info[df_company_info['날짜'].str.startswith(f"{target_year - 1}")]
                if not prev_annual_data.empty:
                    prev_revenue = prev_annual_data['매출액'].sum()
                    if prev_revenue != 0:
                        revenue_growth_rate = round(((revenue - prev_revenue) / prev_revenue) * 100, 2)
                    else:
                        revenue_growth_rate = 0  # prev_revenue가 0인 경우 증가율을 0으로 설정
                else:
                    revenue_growth_rate = 0
            else:
                revenue_growth_rate = 0
            
            # 연도별 데이터 딕셔너리에 추가
            growth_data['year_level_data'][year_key] = {
                '총자산': total_assets,
                '매출액': revenue
            }
            growth_data['year_rate_data'][year_key] = {
                '총자산증가율': asset_growth_rate,
                '매출액증가율': revenue_growth_rate
            }
        else:
            # 데이터가 없는 경우 기본 값 추가
            growth_data['year_level_data'][year_key] = {
                '총자산': 0,
                '매출액': 0
            }
            growth_data['year_rate_data'][year_key] = {
                '총자산증가율': 0,
                '매출액증가율': 0
            }

    # 최신 년월의 데이터도 추가 ('연말 예상'으로 표시, 매출액은 연환산)
    estimate_key = f"{latest_year}년(E)"  # '연말 예상'을 나타내는 키 추가
    # 직전 월의 데이터가 없는 경우, 존재하는 가장 최근의 데이터로 이동
    max_iterations = 12  # 최대 반복 횟수 설정
    iteration_count = 0
    previous_year_month = pd.to_datetime(latest_year_month) - pd.DateOffset(months=1)
    while previous_year_month.strftime("%Y-%m") not in df_company_info['날짜'].values:
        previous_year_month -= pd.DateOffset(months=1)
        iteration_count += 1
        if iteration_count >= max_iterations:
            raise ValueError("Exceeded maximum iterations while searching for previous month data.")
    previous_month_str = previous_year_month.strftime("%Y-%m")
    latest_data = df_company_info[df_company_info['날짜'] == previous_month_str]
    cumulative_data = df_company_info[(pd.to_datetime(df_company_info['날짜']) >= pd.to_datetime(f"{latest_year}-01")) & (pd.to_datetime(df_company_info['날짜']) < pd.to_datetime(latest_year_month))]

    # 최신 데이터를 바탕으로 '연말 예상' 데이터 계산
    if not latest_data.empty and not cumulative_data.empty:
        # 누적 총자산과 매출액을 이용하여 연환산 계산
        cumulative_total_assets = cumulative_data['총자산'].sum()
        total_assets = int(round((cumulative_total_assets / previous_year_month.month)))  # 누적 총자산을 월수로 나누고 12를 곱해 연환산
        cumulative_revenue = cumulative_data['매출액'].sum()
        revenue = int(round((cumulative_revenue / previous_year_month.month) * 12))  # 누적 매출액을 월수로 나누고 12를 곱해 연환산

        # 총자산증가율 계산 (직전 년도 12월 대비)
        prev_year_december_date = f"{latest_year - 1}-12"
        prev_december_data = df_company_info[df_company_info['날짜'] == prev_year_december_date]
        if not prev_december_data.empty:
            prev_total_assets = prev_december_data.at[prev_december_data.index[0], '총자산']
            asset_growth_rate = round(((total_assets - prev_total_assets) / prev_total_assets) * 100, 2)
        else:
            asset_growth_rate = 0

        # 매출액증가율 계산 (전년도 누적 매출액 대비)
        prev_annual_data = df_company_info[df_company_info['날짜'].str.startswith(f"{latest_year - 1}")]
        if not prev_annual_data.empty:
            prev_revenue = prev_annual_data['매출액'].sum()
            if prev_revenue != 0:
                revenue_growth_rate = round(((revenue - prev_revenue) / prev_revenue) * 100, 2)
            else:
                revenue_growth_rate = 0  # prev_revenue가 0인 경우 증가율을 0으로 설정
        else:
            revenue_growth_rate = 0
        
        # 데이터 추가
        growth_data['year_level_data'][estimate_key] = {
            '총자산': total_assets,
            '매출액': revenue
        }
        growth_data['year_rate_data'][estimate_key] = {
            '총자산증가율': asset_growth_rate,
            '매출액증가율': revenue_growth_rate
        }
    else:
        # 데이터가 없는 경우 기본 값 추가
        growth_data['year_level_data'][estimate_key] = {
            '총자산': 0,
            '매출액': 0
        }
        growth_data['year_rate_data'][estimate_key] = {
            '총자산증가율': 0,
            '매출액증가율': 0
        }

    # 'recent_data'에 대해 최신 년월 기준으로 지난 12개월 데이터를 추출
    past_12_months = pd.date_range(end=latest_year_month, periods=12, freq='ME').strftime("%Y-%m").tolist()
    for month in past_12_months:
        # 해당 월의 데이터를 필터링
        monthly_data = df_company_info[df_company_info['날짜'] == month]
        prev_year_month = (pd.to_datetime(month) - pd.DateOffset(years=1)).strftime("%Y-%m")
        prev_monthly_data = df_company_info[df_company_info['날짜'] == prev_year_month]
        recent_12_months_data = df_company_info[(pd.to_datetime(df_company_info['날짜']) <= pd.to_datetime(month)) & (pd.to_datetime(df_company_info['날짜']) > (pd.to_datetime(month) - pd.DateOffset(months=12)))]
        prev_recent_12_months_data = df_company_info[(pd.to_datetime(df_company_info['날짜']) <= pd.to_datetime(prev_year_month)) & (pd.to_datetime(df_company_info['날짜']) > (pd.to_datetime(prev_year_month) - pd.DateOffset(months=12)))]

        if not monthly_data.empty:
            # '성장성' 지표 추출
            revenue = int(round(monthly_data.at[monthly_data.index[0], '매출액']))  # 매출액 (연환산하지 않음)
            prev_revenue = int(round(prev_monthly_data.at[prev_monthly_data.index[0], '매출액'])) if not prev_monthly_data.empty else 0  # 전년 동월 매출액
            revenue_growth_rate = round(((revenue - prev_revenue) / prev_revenue) * 100, 2) if prev_revenue != 0 else 0
            recent_12_months_revenue = int(round(recent_12_months_data['매출액'].sum()))  # 최근 12개월 누적 매출액
            prev_12_months_revenue = int(round(prev_recent_12_months_data['매출액'].sum()))  # 전년 동월 기준 전년 동월 누적 매출액
            cumulative_growth_rate = round(((recent_12_months_revenue - prev_12_months_revenue) / prev_12_months_revenue) * 100, 2) if prev_12_months_revenue != 0 else 0
            
            growth_data['recent_data'][month] = {
                '매출액': revenue,
                '전년동월 매출액': prev_revenue,
                '매출액증가율': revenue_growth_rate,
                '최근12개월 누적 매출액': recent_12_months_revenue,
                '전년동월 누적 매출액': prev_12_months_revenue,
                '누적 매출액증가율': cumulative_growth_rate
            }
        else:
            # 빈 데이터일 경우 기본 값 추가 (예: 0으로 설정)
            growth_data['recent_data'][month] = {
                '매출액': 0,
                '전년동월 매출액': 0,
                '매출액증가율': 0,
                '최근12개월 누적 매출액': 0,
                '전년동월 누적 매출액': 0,
                '누적 매출액증가율': 0
            }
    
    return growth_data


pprint.pprint(preprocess_growth_data(df_company_info[df_company_info['기업명'] == 'Company_1']))

{'latest_year_month': '2024-09',
 'recent_data': {'2023-09': {'누적 매출액증가율': -20.33,
                             '매출액': 2846,
                             '매출액증가율': 36.96,
                             '전년 동월 매출액': 2078,
                             '전년동월 누적 매출액': 76199,
                             '최근 12개월 누적 매출액': 60711},
                 '2023-10': {'누적 매출액증가율': -24.07,
                             '매출액': 3451,
                             '매출액증가율': -37.07,
                             '전년 동월 매출액': 5484,
                             '전년동월 누적 매출액': 77281,
                             '최근 12개월 누적 매출액': 58678},
                 '2023-11': {'누적 매출액증가율': -30.53,
                             '매출액': 3189,
                             '매출액증가율': -63.72,
                             '전년 동월 매출액': 8790,
                             '전년동월 누적 매출액': 76402,
                             '최근 12개월 누적 매출액': 53077},
                 '2023-12': {'누적 매출액증가율': -29.1,
                             '매출액': 9196

In [92]:
def preprocess_profitability_data(df_company_info):
    """
    수익성 지표 데이터를 전처리하여 필요한 JSON 형식의 구조로 변환하는 함수.

    Args:
        df_company_info (DataFrame): 회사 재무 정보 데이터프레임

    Returns:
        dict: 전처리된 수익성 지표 데이터 (JSON 형태)
    """
    # 최신 년월 추출: 데이터프레임에서 가장 최신의 '날짜' 값을 가져옴
    latest_year_month = df_company_info['날짜'].max()
    latest_year = int(latest_year_month.split('-')[0])  # 최신 년도 추출
    latest_month = int(latest_year_month.split('-')[1])  # 최신 월 추출

    # 결과를 저장할 데이터 딕셔너리 정의
    profitability_data = {
        'latest_year_month': latest_year_month,  # 최신 년월 기록
        'year_level_data': {},  # 연간 수준 데이터
        'year_rate_data': {},  # 연간 증가율 데이터
        'recent_data': {}  # 최근 12개월 데이터
    }

    # 'year_level_data' 및 'year_rate_data'에 대해 과거 데이터를 추출
    num_years = 3  # 추출할 과거 연도의 수, 필요에 따라 변경 가능
    for year_offset in range(1, num_years + 1):
        target_year = latest_year - year_offset
        year_key = f"{target_year}년"
        annual_data = df_company_info[df_company_info['날짜'].str.startswith(f"{target_year}")]

        # 연간 데이터를 사용할 수 있는 경우
        if not annual_data.empty:
            # '수익성' 지표 계산
            operating_profit = int(round(annual_data['영업이익'].sum()))  # 해당 연도의 전체 영업이익의 합계
            net_profit = int(round(annual_data['당기순이익'].sum()))  # 해당 연도의 전체 당기순이익의 합계
            revenue = annual_data['매출액'].sum()  # 해당 연도의 매출액 합계

            # 영업이익률과 당기순이익률 계산: 매출액이 있어야 계산 가능
            if revenue != 0:
                operating_profit_margin = round((operating_profit / revenue) * 100, 2)
                net_profit_margin = round((net_profit / revenue) * 100, 2)
            else:
                operating_profit_margin = 0
                net_profit_margin = 0

            # 연도별 데이터 딕셔너리에 추가
            profitability_data['year_level_data'][year_key] = {
                '영업이익': operating_profit,
                '당기순이익': net_profit
            }
            profitability_data['year_rate_data'][year_key] = {
                '영업이익률': operating_profit_margin,
                '당기순이익률': net_profit_margin
            }
        else:
            # 데이터가 없는 경우 기본 값 추가
            profitability_data['year_level_data'][year_key] = {
                '영업이익': 0,
                '당기순이익': 0
            }
            profitability_data['year_rate_data'][year_key] = {
                '영업이익률': 0,
                '당기순이익률': 0
            }

    # 최신 년월의 데이터도 추가 ('연말 예상'으로 표시, 연환산 계산)
    estimate_key = f"{latest_year}년(E)"  # '연말 예상'을 나타내는 키 추가
    max_iterations = 12  # 최대 반복 횟수 설정
    iteration_count = 0
    previous_year_month = pd.to_datetime(latest_year_month) - pd.DateOffset(months=1)
    while previous_year_month.strftime("%Y-%m") not in df_company_info['날짜'].values:
        previous_year_month -= pd.DateOffset(months=1)
        iteration_count += 1
        if iteration_count >= max_iterations:
            raise ValueError("Exceeded maximum iterations while searching for previous month data.")

    previous_month_str = previous_year_month.strftime("%Y-%m")
    latest_data = df_company_info[df_company_info['날짜'] == previous_month_str]
    cumulative_data = df_company_info[(pd.to_datetime(df_company_info['날짜']) >= pd.to_datetime(f"{latest_year}-01")) & (pd.to_datetime(df_company_info['날짜']) < pd.to_datetime(latest_year_month))]

    if not latest_data.empty and not cumulative_data.empty:
        cumulative_operating_profit = cumulative_data['영업이익'].sum()
        cumulative_net_profit = cumulative_data['당기순이익'].sum()
        revenue = cumulative_data['매출액'].sum()

        operating_profit = int(round((cumulative_operating_profit / previous_year_month.month) * 12))
        net_profit = int(round((cumulative_net_profit / previous_year_month.month) * 12))

        if revenue != 0:
            operating_profit_margin = round((operating_profit / revenue) * 100, 2)
            net_profit_margin = round((net_profit / revenue) * 100, 2)
        else:
            operating_profit_margin = 0
            net_profit_margin = 0

        # 데이터 추가
        profitability_data['year_level_data'][estimate_key] = {
            '영업이익': operating_profit,
            '당기순이익': net_profit
        }
        profitability_data['year_rate_data'][estimate_key] = {
            '영업이익률': operating_profit_margin,
            '당기순이익률': net_profit_margin
        }
    else:
        profitability_data['year_level_data'][estimate_key] = {
            '영업이익': 0,
            '당기순이익': 0
        }
        profitability_data['year_rate_data'][estimate_key] = {
            '영업이익률': 0,
            '당기순이익률': 0
        }

    # 'recent_data'에 대해 최신 년월 기준으로 지난 12개월 데이터를 추출
    past_12_months = pd.date_range(end=latest_year_month, periods=12, freq='ME').strftime("%Y-%m").tolist()
    for month in past_12_months:
        monthly_data = df_company_info[df_company_info['날짜'] == month]
        prev_year_month = (pd.to_datetime(month) - pd.DateOffset(years=1)).strftime("%Y-%m")
        prev_monthly_data = df_company_info[df_company_info['날짜'] == prev_year_month]
        recent_12_months_data = df_company_info[(pd.to_datetime(df_company_info['날짜']) <= pd.to_datetime(month)) & (pd.to_datetime(df_company_info['날짜']) > (pd.to_datetime(month) - pd.DateOffset(months=12)))]
        prev_recent_12_months_data = df_company_info[(pd.to_datetime(df_company_info['날짜']) <= pd.to_datetime(prev_year_month)) & (pd.to_datetime(df_company_info['날짜']) > (pd.to_datetime(prev_year_month) - pd.DateOffset(months=12)))]

        if not monthly_data.empty:
            operating_profit = int(round(monthly_data.at[monthly_data.index[0], '영업이익']))
            prev_operating_profit = int(round(prev_monthly_data.at[prev_monthly_data.index[0], '영업이익'])) if not prev_monthly_data.empty else 0
            operating_profit_growth_rate = round(((operating_profit - prev_operating_profit) / prev_operating_profit) * 100, 2) if prev_operating_profit != 0 else 0
            recent_12_months_operating_profit = int(round(recent_12_months_data['영업이익'].sum()))
            prev_12_months_operating_profit = int(round(prev_recent_12_months_data['영업이익'].sum()))
            cumulative_growth_rate = round(((recent_12_months_operating_profit - prev_12_months_operating_profit) / prev_12_months_operating_profit) * 100, 2) if prev_12_months_operating_profit != 0 else 0

            profitability_data['recent_data'][month] = {
                '영업이익': operating_profit,
                '전년동월 영업이익': prev_operating_profit,
                '영업이익증가율': operating_profit_growth_rate,
                '최근12개월 누적 영업이익': recent_12_months_operating_profit,
                '전년동월 누적 영업이익': prev_12_months_operating_profit,
                '누적 영업이익증가율': cumulative_growth_rate
            }
        else:
            profitability_data['recent_data'][month] = {
                '영업이익': 0,
                '전년동월 영업이익': 0,
                '영업이익증가율': 0,
                '최근12개월 누적 영업이익': 0,
                '전년동월 누적 영업이익': 0,
                '누적 영업이익증가율': 0
            }

    return profitability_data


pprint.pprint(preprocess_profitability_data(df_company_info[df_company_info['기업명'] == 'Company_1']))

{'latest_year_month': '2024-09',
 'recent_data': {'2023-09': {'누적 영업이익증가율': -104.23,
                             '영업이익': -352,
                             '영업이익증가율': -208.31,
                             '전년동월 누적 영업이익': -1065,
                             '전년동월 영업이익': 325,
                             '최근12개월 누적 영업이익': 45},
                 '2023-10': {'누적 영업이익증가율': -153.73,
                             '영업이익': 549,
                             '영업이익증가율': -247.58,
                             '전년동월 누적 영업이익': -1798,
                             '전년동월 영업이익': -372,
                             '최근12개월 누적 영업이익': 966},
                 '2023-11': {'누적 영업이익증가율': -88.79,
                             '영업이익': 284,
                             '영업이익증가율': -74.53,
                             '전년동월 누적 영업이익': 1204,
                             '전년동월 영업이익': 1115,
                             '최근12개월 누적 영업이익': 135},
                 '2023-12': {'누적 영업이익증가율': -238.69,
                             '영

In [42]:
# from langchain.callbacks.streaming import StreamingStdOutCallbackHandler

# SQLite 캐시 설정 - 반복된 요청에 대한 비용 절감 목적
# from langchain.cache import SQLiteCache
# langchain_cache = SQLiteCache(database_path="cache.sqlite")

# OpenAI GPT-4 Mini 모델 설정 (LLM 설정)
llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0,
    max_tokens=300,
)

# 디렉토리 설정
CURRENT_DIR = os.getcwd()
PROMPT_PATH = os.path.join(CURRENT_DIR, "prompts")

# prompts 폴더에서 prompt txt 파일 읽기 (보안 걸려서 txt 안 열림)
def load_prompt(file_name):
    """
    주어진 파일 이름의 프롬프트 템플릿을 읽어 반환합니다.
    """
    file_path = os.path.join(PROMPT_PATH, file_name)
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()
    
# 모든 프롬프트 템플릿 로드    
growth_template = load_prompt("growth_template.txt")

# 성장성 지표 분석 프롬프트 설정
growth_prompt = PromptTemplate.from_template(growth_template)

# LCEL 체인 설정
growth_analysis_chain = growth_prompt | llm

In [87]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import FuncFormatter
import plotly.graph_objs as go
import plotly.subplots as sp
from plotly.subplots import make_subplots

plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.family'] = 'Malgun Gothic'

def json_to_dataframe_plotly(json_data: dict):
    '''
    Json data를 DataFrame으로 변경하고 Plotly로 시각화
    '''
    # 'year_level_data'를 DataFrame으로 변환
    year_level_df = pd.DataFrame.from_dict(json_data['year_level_data'], orient='index')
    year_level_df.reset_index(inplace=True)
    year_level_df.rename(columns={'index': '연도'}, inplace=True)
    year_level_df = year_level_df.sort_values(by='연도')  # 연도 순서로 정렬

    # 'year_rate_data'를 DataFrame으로 변환
    year_rate_df = pd.DataFrame.from_dict(json_data['year_rate_data'], orient='index')
    year_rate_df.reset_index(inplace=True)
    year_rate_df.rename(columns={'index': '연도'}, inplace=True)
    year_rate_df = year_rate_df.sort_values(by='연도')  # 연도 순서로 정렬

    # 'recent_data'를 DataFrame으로 변환
    recent_data_df = pd.DataFrame.from_dict(json_data['recent_data'], orient='index')
    recent_data_df.reset_index(inplace=True)
    recent_data_df.rename(columns={'index': '월'}, inplace=True)

    # 1. year_level_data를 가지고 막대그래프 그리기
    fig = make_subplots(rows=1, cols=2, subplot_titles=('연도별 총자산 및 매출액', '연도별 총자산증가율 및 매출액증가율'))

    # 연도별 총자산 및 매출액 (막대 그래프)
    bars = [
        go.Bar(x=year_level_df['연도'], y=year_level_df['총자산'], name='총자산', marker_color='#2a9d8f'),
        go.Bar(x=year_level_df['연도'], y=year_level_df['매출액'], name='매출액', marker_color='#e76f51')
    ]
    fig.add_traces(bars, rows=[1, 1], cols=[1, 1])

    # 연도별 총자산증가율 및 매출액증가율 (라인 그래프)
    lines = [
        go.Scatter(x=year_rate_df['연도'], y=year_rate_df['총자산증가율'], mode='lines+markers', name='총자산증가율', line=dict(color='#264653')),
        go.Scatter(x=year_rate_df['연도'], y=year_rate_df['매출액증가율'], mode='lines+markers', name='매출액증가율', line=dict(color='#f4a261'))
    ]
    fig.add_traces(lines, rows=[1, 1], cols=[2, 2])

    # 레이아웃 설정 및 legend를 각 서브플롯 아래에 배치
    fig.update_layout(
        width=1200,
        height=700,
        showlegend=True,
        legend=dict(
            orientation='h',
            yanchor='top',
            y=-0.12,
            xanchor='center',
            x=0.5
        ),
        annotations=[
            dict(
                text='연도별 총자산 및 매출액',
                x=0.22,
                y=1.02,
                xref='paper',
                yref='paper',
                showarrow=False,
                font=dict(size=20)
            ),
            dict(
                text='연도별 총자산증가율 및 매출액증가율',
                x=0.78,
                y=1.02,
                xref='paper',
                yref='paper',
                showarrow=False,
                font=dict(size=20)
            )
        ]
    )

    fig.show()

    # 2. recent_data를 가지고 월별 데이터 시각화
    fig2 = make_subplots(rows=2, cols=1, subplot_titles=('월별 매출액 및 전년 동월 매출액', '매출액 증가율 및 누적 매출액 증가율'),
                         vertical_spacing=0.08)  # vertical_spacing을 더 줄임

    # 월별 매출액 및 전년 동월 매출액 (막대 그래프)
    bars2 = [
        go.Bar(x=recent_data_df['월'], y=recent_data_df['매출액'], name='매출액', marker_color='#2a9d8f'),
        go.Bar(x=recent_data_df['월'], y=recent_data_df['전년 동월 매출액'], name='전년 동월 매출액', marker_color='#e76f51')
    ]
    fig2.add_traces(bars2, rows=[1, 1], cols=[1, 1])

    # 매출액 증가율 및 누적 매출액 증가율 (라인 그래프)
    lines2 = [
        go.Scatter(x=recent_data_df['월'], y=recent_data_df['매출액증가율'], mode='lines+markers', name='매출액 증가율', line=dict(color='#264653')),
        go.Scatter(x=recent_data_df['월'], y=recent_data_df['누적 매출액증가율'], mode='lines+markers', name='누적 매출액증가율', line=dict(color='#f4a261'))
    ]
    fig2.add_traces(lines2, rows=[2, 2], cols=[1, 1])

    # 레이아웃 설정 및 legend를 각 서브플롯 아래에 배치
    fig2.update_layout(
        width=1200,
        height=900,
        showlegend=True,
        legend=dict(
            orientation='h',
            yanchor='top',
            y=-0.08,
            xanchor='center',
            x=0.5
        ),
        annotations=[
            dict(
                text='월별 매출액 및 전년 동월 매출액',
                x=0.5,
                y=1.02,
                xref='paper',
                yref='paper',
                showarrow=False,
                font=dict(size=20)
            ),
            dict(
                text='매출액 증가율 및 누적 매출액 증가율',
                x=0.5,
                y=0.46,  # y 값을 더 낮춤
                xref='paper',
                yref='paper',
                showarrow=False,
                font=dict(size=20)
            )
        ]
    )

    fig2.show()

    # 피벗된 DataFrame 출력
    year_level_df_pivot = year_level_df.pivot_table(index=None, columns='연도', values=['총자산', '매출액'])
    year_rate_df_pivot = year_rate_df.pivot_table(index=None, columns='연도', values=['총자산증가율', '매출액증가율'])
    recent_data_df_pivot = recent_data_df.pivot_table(index=None, columns='월', values=['매출액', '전년 동월 매출액', '매출액증가율', '최근 12개월 누적 매출액', '전년동월 누적 매출액', '누적 매출액증가율'])

    print("Year Level DataFrame (Pivoted):")
    print(year_level_df_pivot.astype(int))

    print("\nYear Rate DataFrame (Pivoted):")
    print(year_rate_df_pivot)

    print("\nRecent DataFrame (Pivoted):")
    print(recent_data_df_pivot)

In [88]:
pprint.pprint(growth_data)

{'latest_year_month': '2024-09',
 'recent_data': {'2023-09': {'누적 매출액증가율': -20.33,
                             '매출액': 2846,
                             '매출액증가율': 36.96,
                             '전년 동월 매출액': 2078,
                             '전년동월 누적 매출액': 76199,
                             '최근 12개월 누적 매출액': 60711},
                 '2023-10': {'누적 매출액증가율': -24.07,
                             '매출액': 3451,
                             '매출액증가율': -37.07,
                             '전년 동월 매출액': 5484,
                             '전년동월 누적 매출액': 77281,
                             '최근 12개월 누적 매출액': 58678},
                 '2023-11': {'누적 매출액증가율': -30.53,
                             '매출액': 3189,
                             '매출액증가율': -63.72,
                             '전년 동월 매출액': 8790,
                             '전년동월 누적 매출액': 76402,
                             '최근 12개월 누적 매출액': 53077},
                 '2023-12': {'누적 매출액증가율': -29.1,
                             '매출액': 9196

In [89]:
# 10개 회사 sample
company_list = df_company_info['기업명'].unique()[:1]

for firm in company_list:
    print("\n\n",'#'*10, f'{firm}', '#'*10)
    temp = df_company_info[df_company_info['기업명'] == firm]
    
    # 성장성 지표 JSON 데이터 생성
    growth_data = preprocess_growth_data(temp)  # 이전 코드에서 생성된 성장성 지표 데이터
    
    json_to_dataframe_plotly(growth_data)
    
    # 성장성 지표에 대한 분석 요청
    result = growth_analysis_chain.stream({
        "latest_year_month": growth_data["latest_year_month"],
        "year_level_data": growth_data["year_level_data"],
        "year_rate_data": growth_data["year_rate_data"],
        "recent_data": growth_data["recent_data"]
    })

    # Streaming
    print( f"\n{firm}에 대한 AI 답변:")
    stream_response(result)



 ########## Company_1 ##########


Year Level DataFrame (Pivoted):
연도   2021년  2022년  2023년  2024년(E)
매출액  72399  76678  54363     61275
총자산  30687  29873  29656     29669

Year Rate DataFrame (Pivoted):
연도      2021년  2022년  2023년  2024년(E)
매출액증가율  21.16   5.91 -29.10     12.71
총자산증가율  -5.99  -2.65  -0.73      0.04

Recent DataFrame (Pivoted):
월                2023-09   2023-10   2023-11   2023-12   2024-01   2024-02  \
누적 매출액증가율         -20.33    -24.07    -30.53    -29.10    -25.26    -21.30   
매출액              2846.00   3451.00   3189.00   9196.00   4827.00   1605.00   
매출액증가율             36.96    -37.07    -63.72     16.26     16.59    -40.07   
전년 동월 매출액        2078.00   5484.00   8790.00   7910.00   4140.00   2678.00   
전년동월 누적 매출액     76199.00  77281.00  76402.00  76678.00  73655.00  68587.00   
최근 12개월 누적 매출액  60711.00  58678.00  53077.00  54363.00  55050.00  53977.00   

월                2024-03   2024-04   2024-05   2024-06   2024-07   2024-08  
누적 매출액증가율         -26.36    -25.87     -9.46     -8.65     -7.12